In [ ]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
import cv2
from torchvision import transforms
model.eval()

In [ ]:
newmodel = torch.nn.Sequential(*(list(model.children())[:-1]))

In [ ]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [ ]:
image = cv2.imread("sample_images/a0676-kme_609.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
input = cv2.resize(image, (64, 64)) / 255.0
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input).unsqueeze(0).type(torch.float32)
pred =newmodel(input_tensor)

In [ ]:
pred.shape

In [ ]:
image = cv2.imread("sample_images/a0676-kme_609_c.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
input = cv2.resize(image, (64, 64)) / 255.0
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input).unsqueeze(0).type(torch.float32)
target =newmodel(input_tensor)

In [ ]:
from envs.env_dataloader import image_encoder

In [1]:
from envs.photo_env import PhotoEnhancementEnv
import torch
env = PhotoEnhancementEnv(64)

Encoding testing data ...


  0%|          | 0/4 [00:00<?, ?it/s]

finished...
Encoding training data ...


  0%|          | 0/36 [00:00<?, ?it/s]

finished...


In [3]:
obs = env.reset()
parameters = torch.tensor([0.125, 0.125, 0.375, 0.125, 0., 0.0625, 0.9375, 0.375, 0.0625, 0., 0.125, 0.125])
parameters = torch.stack([parameters for i in range(64)])
action = parameters
env.step(action)

({'enhanced_encoded_image': tensor([[3.2310, 4.2118, 4.5486,  ..., 0.0000, 1.6651, 0.0000],
          [4.0241, 0.0000, 1.6446,  ..., 0.9130, 0.0228, 0.0000],
          [1.1111, 0.0000, 0.0088,  ..., 0.0000, 0.9583, 1.9530],
          ...,
          [0.0000, 2.1472, 1.4969,  ..., 2.4731, 3.3409, 0.6564],
          [0.0000, 0.0000, 0.5761,  ..., 0.0220, 0.0000, 0.1313],
          [0.0000, 3.0364, 4.2623,  ..., 2.6856, 2.1538, 0.5002]]),
  'encoded_source': tensor([[0.7249, 0.0000, 0.2135,  ..., 0.0000, 0.0000, 0.0000],
          [0.7249, 0.0000, 0.2135,  ..., 0.0000, 0.0000, 0.0000],
          [0.7249, 0.0000, 0.2135,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.7249, 0.0000, 0.2135,  ..., 0.0000, 0.0000, 0.0000],
          [0.7249, 0.0000, 0.2135,  ..., 0.0000, 0.0000, 0.0000],
          [0.7249, 0.0000, 0.2135,  ..., 0.0000, 0.0000, 0.0000]]),
  'source_image': tensor([[[[  3,   3,   5,  ...,  32,  32,  32],
            [  6,   4,   3,  ...,  32,  32,  32],
            [ 

In [ ]:
f= torch.rand(32,3,64,64)
index = torch.tensor([1,3,4])
a= torch.index_select(f,0,index)

In [ ]:
torch.Tensor([int(index) for index in range(3)]).to(torch.int32)

In [ ]:
from envs.new_edit_photo import PhotoEditor

editor = PhotoEditor()
parameters = torch.tensor([0.125, 0.125, 0.375, 0.125, 0., 0.0625, 0.9375, 0.375, 0.0625, 0., 0.125, 0.125])
parameters = torch.stack([parameters for i in range(64)])

In [ ]:
for i,j,k,l in train:
    # print(i.dtype)
    enhanced = i.permute(0,2,3,1)/255.0
    # print(enhanced.dtype)
    enhanced = editor(enhanced,parameters)
    enhanced = enhanced.permute(0,3,1,2)
    enhanced =torch.flatten(enhanced,start_dim=1, end_dim=-1)

    target = torch.flatten(j,start_dim=1, end_dim=-1)/255.0
    # print(enhanced.dtype)
    rmse = enhanced-target
    rmse = torch.pow(rmse,2).mean(1)
    rmse = torch.sqrt(rmse)
    break

print(rmse.shape)